# TFX Components Walk-through

## Learning objectives

1.  Develop a high level understanding of TFX pipeline components.
2.  Learn how to use a TFX Interactive Context for prototype development of TFX pipelines.
3.  Work with the Tensorflow Data Validation (TFDV) library to check and analyze input data.
4.  Utilize the Tensorflow Transform (TFT) library for scalable data preprocessing and feature transformations.
5.  Employ the Tensorflow Model Analysis (TFMA) library for model evaluation.

## Introduction

In this notebook, you will work with the [Covertype Data Set](https://github.com/jarokaz/mlops-labs/blob/master/datasets/covertype/README.md) and use TFX to analyze, understand, and pre-process the dataset and train, analyze, validate, and deploy a multi-class classification model to predict the type of forest cover from cartographic features.

You will utilize  **TFX Interactive Context** to work with the TFX components interactivelly in a Jupyter notebook environment. Working in an interactive notebook is useful when doing initial data exploration, experimenting with models, and designing ML pipelines. You should be aware that there are differences in the way interactive notebooks are orchestrated, and how they access metadata artifacts. In a production deployment of TFX on GCP, you will use an orchestrator such as Kubeflow Pipelines, or Cloud Composer. In an interactive mode, the notebook itself is the orchestrator, running each TFX component as you execute the notebook cells. In a production deployment, ML Metadata will be managed in a scalabe database like MySQL, and artifacts in apersistent store such as Google Cloud Storage. In an interactive mode, both properties and payloads are stored in a local file system of the Jupyter host.

**Setup Note**:

Currently, TFMA visualizations do not render properly in JupyterLab. It is recommended to run this notebook in Jupyter Classic Notebook. To switch to Classic Notebook select **Launch Classic Notebook** from the **Help** menu.

In [1]:
import sys
print(sys.version)

3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]


In [2]:
#!pip install tfx

In [3]:
import absl
import os
import tempfile
import time

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tfx

from pprint import pprint
from tensorflow_metadata.proto.v0 import schema_pb2, statistics_pb2, anomalies_pb2
from tensorflow_transform.tf_metadata import schema_utils
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import InfraValidator
from tfx.components import Pusher
#from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components import Tuner
from tfx.dsl.components.base import executor_spec
#from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.components.trainer import executor as trainer_executor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import infra_validator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import HyperParameters
from tfx.types.standard_artifacts import ModelBlessing
from tfx.types.standard_artifacts import InfraBlessing

2025-03-05 16:46:13.693190: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 16:46:13.953220: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 16:46:14.286770: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-05 16:46:14.570073: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-05 16:46:14.571846: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 16:46:15.049870: I tensorflow/core/platform/cpu_feature_guard.cc:

In [4]:
#!pip install tensorflow

In [5]:
print("Tensorflow Version:", tf.__version__) # done with version 2.16.2
print("TFX Version:", tfx.__version__) # done with version 1.16.0
print("TFDV Version:", tfdv.__version__) # done with version 1.16.1
print("TFMA Version:", tfma.VERSION_STRING) # done with version 0.47.1

absl.logging.set_verbosity(absl.logging.INFO)

Tensorflow Version: 2.16.2
TFX Version: 1.16.0
TFDV Version: 1.16.1
TFMA Version: 0.47.1


If the versions above do not match, update your packages in the current Jupyter kernel below. The default `%pip` package installation location is not on your system installation PATH; use the command below to append the local installation path to pick up the latest package versions. Note that you may also need to restart your notebook kernel to pick up the specified package versions and re-run the imports cell above before proceeding with the lab.

In [6]:
os.environ['PATH'] += os.pathsep + '/home/jupyter/.local/bin'

(Optional) Uncomment and run the cell below only if your package versions do not match the lab defaults.

In [7]:
#!python -v

In [1]:
# %pip install --upgrade --user tensorflow==2.16.2
# %pip install --upgrade --user tfx==1.16.0
# %pip install --upgrade --user tensorflow_data_validation==1.16.0
# %pip install --upgrade --user tensorflow_model_analysis==0.47.1

**Restart the kernel by using Kernel > Restart kernel > Restart.**

## Configure lab settings

Set constants, location paths and other environment settings. 

In [10]:
import absl
import os
import tempfile
import time

import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tfx

from pprint import pprint
from tensorflow_metadata.proto.v0 import schema_pb2, statistics_pb2, anomalies_pb2
from tensorflow_transform.tf_metadata import schema_utils
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import InfraValidator
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components import Tuner
from tfx.dsl.components.base import executor_spec
from tfx.dsl.components.common.importer import Importer
from tfx.v1.dsl import Resolver
from tfx.components.trainer import executor as trainer_executor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import infra_validator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tensorflow.core.framework import graph_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import HyperParameters
from tfx.types.standard_artifacts import ModelBlessing
from tfx.types.standard_artifacts import InfraBlessing

In [11]:
ARTIFACT_STORE = os.path.join(os.sep, 'home', 'jupyter', 'artifact-store')
SERVING_MODEL_DIR=os.path.join(os.sep, 'home', 'jupyter', 'serving_model')
DATA_ROOT = 'gs://cloud-training/OCBL203/workshop-datasets'

## Creating Interactive Context

TFX Interactive Context allows you to create and run TFX Components in an interactive mode. It is designed to support experimentation and development in a Jupyter Notebook environment. It is an experimental feature and major changes to interface and functionality are expected. When creating the interactive context you can specifiy the following parameters:
- `pipeline_name` - Optional name of the pipeline for ML Metadata tracking purposes. If not specified, a name will be generated for you.
- `pipeline_root` - Optional path to the root of the pipeline's outputs. If not specified, an ephemeral temporary directory will be created and used.
- `metadata_connection_config` - Optional `metadata_store_pb2.ConnectionConfig` instance used to configure connection to a ML Metadata connection. If not specified, an ephemeral SQLite MLMD connection contained in the pipeline_root directory with file name "metadata.sqlite" will be used.


In [12]:
PIPELINE_NAME = 'tfx-covertype-classifier'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME, time.strftime("%Y%m%d_%H%M%S"))
os.makedirs(PIPELINE_ROOT, exist_ok=True)

context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=None)

## Ingesting data using ExampleGen

In any ML development process the first step  is to ingest the training and test datasets. The `ExampleGen` component ingests data into a TFX pipeline. It consumes external files/services to generate a set file files in the `TFRecord` format,  which will be used by other TFX components. It can also shuffle the data and split into an arbitrary number of partitions.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/ExampleGen.png width="300">

### Configure and run CsvExampleGen

In this exercise, you use the `CsvExampleGen` specialization of `ExampleGen` to ingest CSV files from a GCS location and emit them as `tf.Example` records for consumption by downstream TFX pipeline components. Your task is to configure the component to create 80-20 `train` and `eval` splits. *Hint: review the [ExampleGen proto](https://github.com/tensorflow/tfx/blob/master/tfx/proto/example_gen.proto#L78) definition to split your data with hash buckets.*

In [13]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        # TODO: Your code to configure train data split
        # TODO: Your code to configure eval data split
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4), # solution
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1) # solution
    ]))

example_gen = tfx.components.CsvExampleGen(
    input_base=DATA_ROOT,
    output_config=output_config)

In [14]:
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data gs://cloud-training/OCBL203/workshop-datasets/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Examine the ingested data

In [15]:
examples_uri = example_gen.outputs['examples'].get()[0].uri
tfrecord_filenames = [os.path.join(examples_uri, 'Split-train', name)
                      for name in os.listdir(os.path.join(examples_uri, 'Split-train'))]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
  example = tf.train.Example()
  example.ParseFromString(tfrecord.numpy())
  for name, feature in example.features.feature.items():
    if feature.HasField('bytes_list'):
        value = feature.bytes_list.value
    if feature.HasField('float_list'):
        value = feature.float_list.value
    if feature.HasField('int64_list'):
        value = feature.int64_list.value
    print('{}: {}'.format(name, value))
  print('******')

Hillshade_9am: [223]
Elevation: [3142]
Hillshade_Noon: [247]
Cover_Type: [1]
Horizontal_Distance_To_Fire_Points: [1871]
Horizontal_Distance_To_Hydrology: [648]
Horizontal_Distance_To_Roadways: [757]
Soil_Type: [b'C7757']
Slope: [9]
Vertical_Distance_To_Hydrology: [101]
Wilderness_Area: [b'Commanche']
Hillshade_3pm: [157]
Aspect: [183]
******
Horizontal_Distance_To_Fire_Points: [451]
Cover_Type: [2]
Aspect: [124]
Vertical_Distance_To_Hydrology: [9]
Hillshade_9am: [245]
Hillshade_3pm: [105]
Slope: [16]
Horizontal_Distance_To_Hydrology: [60]
Hillshade_Noon: [227]
Elevation: [1967]
Soil_Type: [b'C2704']
Wilderness_Area: [b'Cache']
Horizontal_Distance_To_Roadways: [124]
******


2025-03-05 16:47:10.291626: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Generating statistics using StatisticsGen

The `StatisticsGen`  component generates data statistics that can be used by other TFX components. StatisticsGen uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started). `StatisticsGen` generate statistics for each split in the `ExampleGen` component's output. In our case there two splits: `train` and `eval`.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/StatisticsGen.png width="200">

### Configure and  run the `StatisticsGen` component

In [16]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

INFO:absl:Excluding no splits because exclude_splits is not set.


In [17]:
context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Visualize statistics

The generated statistics can be visualized using the `tfdv.visualize_statistics()` function from the [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library or using a utility method of the `InteractiveContext` object. In fact, most of the artifacts generated by the TFX components can be visualized using `InteractiveContext`.

In [18]:
context.show(statistics_gen.outputs['statistics'])

## Infering data schema using SchemaGen

Some TFX components use a description input data called a schema. The schema is an instance of `schema.proto`. It can specify data types for feature values, whether a feature has to be present in all examples, allowed value ranges, and other properties. `SchemaGen` automatically generates the schema by inferring types, categories, and ranges from data statistics. The auto-generated schema is best-effort and only tries to infer basic properties of the data. It is expected that developers review and modify it as needed. `SchemaGen` uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

The `SchemaGen` component generates the schema using the statistics for the `train` split. The statistics for other splits are ignored.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/SchemaGen.png width="200">

### Configure and run the `SchemaGen` components

In [19]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)

INFO:absl:Excluding no splits because exclude_splits is not set.


In [20]:
context.run(schema_gen)

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Visualize the inferred schema

In [21]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,-
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## Updating the auto-generated schema

In most cases the auto-generated schemas must be fine-tuned manually using insights from data exploration and/or domain knowledge about the data. For example, you know that in the `covertype` dataset there are seven types of forest cover (coded using 1-7 range) and that the value of the `Slope` feature should be in the 0-90 range. You can manually add these constraints to the auto-generated schema by setting the feature domain.

### Load the auto-generated schema proto file

In [22]:
schema_proto_path = '{}/{}'.format(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_proto_path)

### Modify the schema

You can use the protocol buffer APIs to modify the schema. 

**Hint**: Review the [TFDV library API documentation](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/set_domain) on setting a feature's domain. You can use the protocol buffer APIs to modify the schema. Review the [Tensorflow Metadata proto definition](https://github.com/tensorflow/metadata/blob/master/tensorflow_metadata/proto/v0/schema.proto#L405) for configuration options.

In [23]:
# TODO: Your code to restrict the categorical feature Cover_Type between the values of 0 and 6.
tfdv.set_domain(
    schema=schema,
    feature_path= "Cover_Type",
    domain= schema_pb2.IntDomain(min=0, max=6)
)
# TODO: Your code to restrict the numeric feature Slope between 0 and 90.

tfdv.set_domain(
    schema=schema,
    feature_path= "Slope",
    domain= schema_pb2.IntDomain(min=0, max=90)
)

tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,min: 0; max: 6
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


#### Save the updated schema 

In [24]:
schema_dir = os.path.join(ARTIFACT_STORE, 'schema')
tf.io.gfile.makedirs(schema_dir)
schema_file = os.path.join(schema_dir, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

feature {
  name: "Aspect"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Cover_Type"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  int_domain {
    min: 0
    max: 6
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Elevation"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Hillshade_3pm"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Hillshade_9am"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Hillshade_Noon"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Horizontal_Distance_To_Fire_Points"
  value_count {
    min: 1
    max: 1
  }

## Importing the updated schema using Importer

The `Importer` component allows you to import an external artifact, including the schema file, so it can be used by other TFX components in your workflow. 


### Configure and run the `Importer` component

In [25]:
schema_importer = Importer(
    source_uri=schema_dir,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=False)

In [26]:
context.run(schema_importer)

INFO:absl:Running driver for Importer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: /home/jupyter/artifact-store/schema, properties: {}, custom_properties: {}
INFO:absl:Running executor for Importer
INFO:absl:Running publisher for Importer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Importer
    execution_id: 4
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 4
        type_id: 18
        uri: "/home/jupyter/artifact-store/schema"
        custom_properties {
          key: "is_external"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.16.0"
          }
        }
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
            _input_trigger: None
        ))

### Visualize the imported schema

In [27]:
context.show(schema_importer.outputs['result'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,min: 0; max: 6
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## Validating data with ExampleValidator

The `ExampleValidator` component identifies anomalies in data.  It identifies anomalies by comparing data statistics computed by the `StatisticsGen` component against a schema generated by `SchemaGen` or imported by `ImporterNode`.

`ExampleValidator` can detect different classes of anomalies. For example it can:

- perform validity checks by comparing data statistics against a schema 
- detect training-serving skew by comparing training and serving data.
- detect data drift by looking at a series of data.


The `ExampleValidator` component validates the data in the `eval` split only. Other splits are ignored. 

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/ExampleValidator.png width="350">

### Configure and run the `ExampleValidator` component


In [28]:
# TODO: Complete ExampleValidator
# Hint: review the visual above and review the documentation on ExampleValidator's inputs and outputs: 
# https://www.tensorflow.org/tfx/guide/exampleval
# Make sure you use the output of the schema_importer component created above.
example_validator = ExampleValidator(
statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result']
)


INFO:absl:Excluding no splits because exclude_splits is not set.


In [29]:
if     len(example_validator.outputs['anomalies'].get())>0:
    train_uri = example_validator.outputs['anomalies'].get()[0].uri
    train_anomalies_filepath=os.path.join(train_uri,"Split-train/SchemaDiff.pb")

    anomalies = anomalies_pb2.Anomalies()

    with open(train_anomalies_filepath, 'rb') as f:
        anomalies_bytes = f.read()
        anomalies.ParseFromString(anomalies_bytes)


    if anomalies.anomaly_info:
        print(tfdv.display_anomalies(anomalies))
    else:
        print("No anomalies found in the dataset")
    
else :
    print("No anomalies found in the dataset")




No anomalies found in the dataset


### Visualize validation results

The anomalies file can be visualized using `context.show`.

In [30]:
context.show(example_validator.outputs['anomalies'])

In our case no anomalies were detected in the `eval` split.

For a detailed deep dive into data validation and schema generation refer to the `lab-31-tfdv-structured-data` lab.

## Preprocessing data with Transform

The `Transform` component performs data transformation and feature engineering. The `Transform` component consumes `tf.Examples` emitted from the `ExampleGen` component and emits the transformed feature data and the `SavedModel` graph that was used to process the data. The emitted `SavedModel`  can then be used by serving components to make sure that the same data pre-processing logic is applied at training and serving.

The `Transform` component requires more code than many other components because of the arbitrary complexity of the feature engineering that you may need for the data and/or model that you're working with. It requires code files to be available which define the processing needed.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Transform.png width="400">

### Define the pre-processing module

To configure `Trainsform`, you need to encapsulate your pre-processing code in the Python `preprocessing_fn` function and save it to a  python module that is then provided to the Transform component as an input. This module will be loaded by transform and the `preprocessing_fn` function will be called when the `Transform` component runs.

In most cases, your implementation of the `preprocessing_fn` makes extensive use of [TensorFlow Transform](https://www.tensorflow.org/tfx/guide/tft) for performing feature engineering on your dataset.

In [31]:
TRANSFORM_MODULE = 'preprocessing.py'

### Configure and run the `Transform` component.

In [32]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=TRANSFORM_MODULE)

In [33]:
context.run(transform)

INFO:absl:Generating ephemeral wheel package for '/home/jupyter/mlops-on-gcp/workshops/tfx-caip-tf23/lab-01-tfx-walkthrough/labs/preprocessing.py' (including modules: ['features', 'preprocessing', 'model']).
INFO:absl:User module package has hash fingerprint version 9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f.
INFO:absl:Executing: ['/opt/conda/bin/python', '/var/tmp/tmpv6tonqyf/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/var/tmp/tmpg_l9933i', '--dist-dir', '/var/tmp/tmp2h9tvmxt']


running bdist_wheel
running build
running build_py
creating build/lib
copying features.py -> build/lib
copying preprocessing.py -> build/lib
copying model.py -> build/lib
installing to /var/tmp/tmpg_l9933i
running install
running install_lib


/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:79: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
INFO:absl:Successfully built user code wheel distribution at '/home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/_wheels/tfx_user_code_Transform-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f-py3-none-any.whl'; target user module is 'preprocessing'.
INFO:absl:Full user module path is 'preprocessing@/home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/_wheels/

copying build/lib/features.py -> /var/tmp/tmpg_l9933i/.
copying build/lib/model.py -> /var/tmp/tmpg_l9933i/.
copying build/lib/preprocessing.py -> /var/tmp/tmpg_l9933i/.
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /var/tmp/tmpg_l9933i/./tfx_user_code_Transform-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f-py3.10.egg-info
running install_scripts
creating /var/tmp/tmpg_l9933i/tfx_user_code_Transform-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f.dist-info/


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
INFO:absl:Successfully installed '/home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/_wheels/tfx_user_code_Transform-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f-py3-none-any.whl'.
INFO:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'preprocessing@/home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/_wheels/tfx_user_code_Transform-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f-py3-none-any.whl', 'stats_options_updater_fn': None} 'stats_options_updater_fn'
INFO:absl:Installing '/home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/_wheels/tfx_user_code_Transform-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/opt/conda/bin/python', '-m', 'pip', 'install', '--target', '/var/tmp/tmp_o

...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Examine the `Transform` component's outputs

The Transform component has 2 outputs:

- `transform_graph` - contains the graph that can perform the preprocessing operations (this graph will be included in the serving and evaluation models).
- `transformed_examples` - contains the preprocessed training and evaluation data.

Take a peek at the `transform_graph` artifact: it points to a directory containing 3 subdirectories:

In [34]:
os.listdir(transform.outputs['transform_graph'].get()[0].uri)

['transformed_metadata', 'metadata', 'transform_fn']

And the `transform.examples` artifact

In [35]:
os.listdir(transform.outputs['transformed_examples'].get()[0].uri)

['Split-eval', 'Split-train']

In [36]:
transform_uri = transform.outputs['transformed_examples'].get()[0].uri
tfrecord_filenames = [os.path.join(transform_uri,  'Split-train', name)
                      for name in os.listdir(os.path.join(transform_uri, 'Split-train'))]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
  example = tf.train.Example()
  example.ParseFromString(tfrecord.numpy())
  for name, feature in example.features.feature.items():
    if feature.HasField('bytes_list'):
        value = feature.bytes_list.value
    if feature.HasField('float_list'):
        value = feature.float_list.value
    if feature.HasField('int64_list'):
        value = feature.int64_list.value
    print('{}: {}'.format(name, value))
  print('******')

Vertical_Distance_To_Hydrology_xf: [0.950199544429779]
Soil_Type_xf: [3]
Horizontal_Distance_To_Fire_Points_xf: [-0.08278580754995346]
Horizontal_Distance_To_Roadways_xf: [-1.0252416133880615]
Aspect_xf: [0.24982306361198425]
Hillshade_9am_xf: [0.4025527536869049]
Hillshade_Noon_xf: [1.1993097066879272]
Wilderness_Area_xf: [1]
Slope_xf: [-0.6797884106636047]
Cover_Type_xf: [1]
Horizontal_Distance_To_Hydrology_xf: [1.7856396436691284]
Hillshade_3pm_xf: [0.38384684920310974]
Elevation_xf: [0.651531994342804]
******
Wilderness_Area_xf: [2]
Horizontal_Distance_To_Hydrology_xf: [-0.9832831621170044]
Elevation_xf: [-3.5473971366882324]
Vertical_Distance_To_Hydrology_xf: [-0.6439016461372375]
Hillshade_Noon_xf: [0.1884450614452362]
Soil_Type_xf: [19]
Horizontal_Distance_To_Roadways_xf: [-1.4311398267745972]
Hillshade_9am_xf: [1.2273873090744019]
Slope_xf: [0.25757989287376404]
Aspect_xf: [-0.2777790129184723]
Cover_Type_xf: [2]
Horizontal_Distance_To_Fire_Points_xf: [-1.1525152921676636]
Hill

2025-03-05 16:48:08.404583: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Train your TensorFlow model with the `Trainer` component

The `Trainer` component trains a model using TensorFlow.

`Trainer` takes:

- tf.Examples used for training and eval.
- A user provided module file that defines the trainer logic.
- A data schema created by `SchemaGen` or imported by `ImporterNode`.
- A proto definition of train args and eval args.
- An optional transform graph produced by upstream Transform component.
- An optional base models used for scenarios such as warmstarting training.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Trainer.png width="400">


### Define the trainer module

To configure `Trainer`, you need to encapsulate your training code in a Python module that is then provided to the `Trainer` as an input. 


In [37]:
TRAINER_MODULE_FILE = 'model.py'

### Create and run the Trainer component

As of the 0.25.0 release of TFX, the `Trainer` component only supports passing a single field - `num_steps` - through the `train_args` and `eval_args` arguments. 

In [38]:
trainer = Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=TRAINER_MODULE_FILE,
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_importer.outputs['result'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1000))

In [39]:
context.run(trainer)

INFO:absl:Generating ephemeral wheel package for '/home/jupyter/mlops-on-gcp/workshops/tfx-caip-tf23/lab-01-tfx-walkthrough/labs/model.py' (including modules: ['features', 'preprocessing', 'model']).
INFO:absl:User module package has hash fingerprint version 9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f.
INFO:absl:Executing: ['/opt/conda/bin/python', '/var/tmp/tmpzxweayu7/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/var/tmp/tmpovdvi0dh', '--dist-dir', '/var/tmp/tmpfaxzy4ya']


running bdist_wheel
running build
running build_py
creating build/lib
copying features.py -> build/lib
copying preprocessing.py -> build/lib
copying model.py -> build/lib
installing to /var/tmp/tmpovdvi0dh
running install
running install_lib
copying build/lib/features.py -> /var/tmp/tmpovdvi0dh/.
copying build/lib/model.py -> /var/tmp/tmpovdvi0dh/.
copying build/lib/preprocessing.py -> /var/tmp/tmpovdvi0dh/.
running install_egg_info
running egg_info


/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:79: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
INFO:absl:Successfully built user code wheel distribution at '/home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/_wheels/tfx_user_code_Trainer-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f-py3-none-any.whl'; target user module is 'model'.
INFO:absl:Full user module path is 'model@/home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/_wheels/tfx_user_code_Trai

creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /var/tmp/tmpovdvi0dh/./tfx_user_code_Trainer-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f-py3.10.egg-info
running install_scripts
creating /var/tmp/tmpovdvi0dh/tfx_user_code_Trainer-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f.dist-info/WHEEL
creating '/var/tmp/tmpfaxzy4ya/tfx_user_code_Trainer-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f-py3-none-any.whl' and adding '/var/tmp/tmpovdvi0dh' to it
adding 'features.py'
adding 'model.py'
adding 


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
INFO:absl:Successfully installed '/home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/_wheels/tfx_user_code_Trainer-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f-py3-none-any.whl'.
INFO:absl:Training model.
INFO:absl:Feature Aspect_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Cover_Type_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Elevation_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_3pm_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_9am_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_Noon_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Fire_Points_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Hydrology_xf has a shape . Setting to DenseTensor.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:absl:Feature Aspect has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Cover_Type has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Elevation has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Hillshade_3pm has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Hillshade_9am has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Hillshade_Noon has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Horizontal_Distance_To_Fire_Points has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Horizontal_Distance_To_Hydrology has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Horizontal_Distance_To_Roadways has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Slope has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Soil_Type has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Vertical_Distance_To_Hydrology

INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/Trainer/model/6/Format-Serving/assets


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/Trainer/model/6/Format-Serving/assets
INFO:absl:Writing fingerprint to /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/Trainer/model/6/Format-Serving/fingerprint.pb
INFO:absl:Model saved successfully
INFO:absl:Training complete. Model written to /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/Trainer/model/6/Format-Serving. ModelRun written to /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/Trainer/model_run/6
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Analyzing training runs with TensorBoard

In this step you will analyze the training run with [TensorBoard.dev](https://blog.tensorflow.org/2019/12/introducing-tensorboarddev-new-way-to.html). `TensorBoard.dev` is a managed service that enables you to easily host, track and share your ML experiments.

#### Retrieve the location of TensorBoard logs

Each model run's train and eval metric logs are written to the `model_run` directory by the Tensorboard callback defined in `model.py`.

In [40]:
logs_path = trainer.outputs['model_run'].get()[0].uri
print(logs_path)

/home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/Trainer/model_run/6


#### Upload the logs and start TensorBoard.dev

1. Open a new JupyterLab terminal window

2. From the terminal window, execute the following command
```
tensorboard dev upload --logdir [YOUR_LOGDIR]
```

Where [YOUR_LOGDIR] is an URI retrieved by the previous cell.

You will be asked to authorize `TensorBoard.dev` using your Google account. If you don't have a Google account or you don't want to authorize `TensorBoard.dev` you can skip this exercise.

After the authorization process completes, follow the link provided to view your experiment.

## Tune your model's hyperparameters with the `Tuner` component

The `Tuner` component makes use of the Python [KerasTuner](https://keras-team.github.io/keras-tuner/) API to tune your model's hyperparameters. It tighty integrates with the `Transform` and `Trainer` components for model hyperparameter tuning in continuous training pipelines as well as advanced use cases such as feature selection, feature engineering, and model architecture search.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Tuner_Overview.png width="400">

`Tuner` takes:

*  A user provided module file (or module fn) that defines the tuning logic, including model definition, hyperparameter search space, objective etc.
*  tf.Examples used for training and eval.
*  Protobuf definition of train args and eval args.
  * (Optional) Protobuf definition of tuning args.
  * (Optional) transform graph produced by an upstream Transform component.
  * (Optional) A data schema created by a SchemaGen pipeline component and optionally altered by the developer.
  
<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Tuner.png width="400">

With the given data, model, and objective, `Tuner` tunes the hyperparameters and emits the best results that can be directly fed into the `Trainer` component during model re-training.

In [41]:
tuner = Tuner(
        module_file=TRAINER_MODULE_FILE,
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        train_args=trainer_pb2.TrainArgs(num_steps=1000),
        eval_args=trainer_pb2.EvalArgs(num_steps=500))

In [43]:
context.run(tuner)

INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.001, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}}, {'class_name': 'Int', 'config': {'name': 'n_layers', 'default': 1, 'conditions': [], 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'n_units_1', 'default': 8, 'conditions': [{'class_name': 'Parent', 'config': {'name': 'n_layers', 'values': [1]}}], 'min_value': 8, 'max_value': 128, 'step': 8, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'n_units_1', 'default': 8, 'conditions': [{'class_name': 'Parent', 'config': {'name': 'n_layers', 'values': [2]}}], 'min_value': 8, 'max_value': 128, 'step': 8, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'n_units_2', 'default': 8, 'conditions': [{'class_name': 'Parent', 'config': {'name': 'n_layers', 'values': [2]}}], '

Trial 5 Complete [00h 00m 01s]

Best val_sparse_categorical_accuracy So Far: 0.7201874852180481
Total elapsed time: 00h 00m 17s
Results summary
Results in /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/.temp/8/covertype_tuning
Showing 10 best trials
Objective(name="val_sparse_categorical_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
learning_rate: 0.001
n_layers: 2
n_units_2: 64
n_units_1: 48
Score: 0.7201874852180481

Trial 1 summary
Hyperparameters:
learning_rate: 0.001
n_layers: 2
n_units_2: 112
n_units_1: 16
Score: 0.7139687538146973

Trial 0 summary
Hyperparameters:
learning_rate: 0.0001
n_layers: 1
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/opt/conda/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
   

ExecutionResult(
    component_id: Tuner
    execution_id: 8
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Retrain your model by running Tuner with the best hyperparameters

In [44]:
hparams_importer = Importer(
    # This can be Tuner's output file or manually edited file. The file contains
    # text format of hyperparameters (kerastuner.HyperParameters.get_config())
    source_uri=tuner.outputs['best_hyperparameters'].get()[0].uri,
    artifact_type=HyperParameters)

In [45]:
context.run(hparams_importer)

INFO:absl:Running driver for Importer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/Tuner/best_hyperparameters/8, properties: {}, custom_properties: {}
INFO:absl:Reusing existing artifact
INFO:absl:Running executor for Importer
INFO:absl:Running publisher for Importer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Importer
    execution_id: 9
    outputs:
        result: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 17
        type_id: 28
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/Tuner/best_hyperparameters/8"
        custom_properties {
          key: "name"
          value {
            string_value: "best_hyperparameters:2025-03-05T16:49:29.972345"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Tuner"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.16.0"
          }
        }
        state: LIVE
        name: "best_hyperparameters:2025-03-05T16:49:29.972345"
        type: "HyperParameters"
        create_time_since_epoch: 1741193370010
        last_update_time_since_epoch: 1741193389250
        , artifact_type: id: 28
        name: "HyperParameters"
        )]
            additional_properties: {}
            additional_custom_properties: {}
            _input_trigger: None
        ))

In [46]:
# TODO: your code to retrain your model with the best hyperparameters found by the Tuner component above.
# Hint: review the Trainer code above in this notebook and the documentation for how to configure the trainer 
# to use the output artifact from the hparams_importer.
trainer = Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=TRAINER_MODULE_FILE,
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_importer.outputs['result'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1000),
    hyperparameters = hparams_importer.outputs['result'] #solution
)


In [47]:
context.run(trainer)

INFO:absl:Generating ephemeral wheel package for '/home/jupyter/mlops-on-gcp/workshops/tfx-caip-tf23/lab-01-tfx-walkthrough/labs/model.py' (including modules: ['features', 'preprocessing', 'model']).
INFO:absl:User module package has hash fingerprint version 9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f.
INFO:absl:Executing: ['/opt/conda/bin/python', '/var/tmp/tmpk9pqgwne/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/var/tmp/tmpko31fbps', '--dist-dir', '/var/tmp/tmp7xkuuymr']


running bdist_wheel
running build
running build_py
creating build/lib
copying features.py -> build/lib
copying preprocessing.py -> build/lib
copying model.py -> build/lib
installing to /var/tmp/tmpko31fbps
running install
running install_lib
copying build/lib/features.py -> /var/tmp/tmpko31fbps/.
copying build/lib/model.py -> /var/tmp/tmpko31fbps/.
copying build/lib/preprocessing.py -> /var/tmp/tmpko31fbps/.
running install_egg_info
running egg_info


/opt/conda/lib/python3.10/site-packages/setuptools/_distutils/cmd.py:79: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
INFO:absl:Successfully built user code wheel distribution at '/home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/_wheels/tfx_user_code_Trainer-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f-py3-none-any.whl'; target user module is 'model'.
INFO:absl:Full user module path is 'model@/home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/_wheels/tfx_user_code_Trai

creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /var/tmp/tmpko31fbps/./tfx_user_code_Trainer-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f-py3.10.egg-info
running install_scripts
creating /var/tmp/tmpko31fbps/tfx_user_code_Trainer-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f.dist-info/WHEEL
creating '/var/tmp/tmp7xkuuymr/tfx_user_code_Trainer-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f-py3-none-any.whl' and adding '/var/tmp/tmpko31fbps' to it
adding 'features.py'
adding 'model.py'
adding 


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
INFO:absl:Successfully installed '/home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/_wheels/tfx_user_code_Trainer-0.0+9e0c802d65bf92c5e31734a1f8f176e2ccd1ec5b3e9a70cf07f1853d9c2d354f-py3-none-any.whl'.
INFO:absl:Training model.
INFO:absl:Feature Aspect_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Cover_Type_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Elevation_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_3pm_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_9am_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_Noon_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Fire_Points_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Hydrology_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_D

...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:absl:Feature Aspect has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Cover_Type has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Elevation has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Hillshade_3pm has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Hillshade_9am has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Hillshade_Noon has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Horizontal_Distance_To_Fire_Points has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Horizontal_Distance_To_Hydrology has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Horizontal_Distance_To_Roadways has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Slope has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Soil_Type has no shape. Setting to varlen_sparse_tensor.
INFO:absl:Feature Vertical_Distance_To_Hydrology

INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/Trainer/model/10/Format-Serving/assets


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/Trainer/model/10/Format-Serving/assets
INFO:absl:Writing fingerprint to /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/Trainer/model/10/Format-Serving/fingerprint.pb
INFO:absl:Model saved successfully
INFO:absl:Training complete. Model written to /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/Trainer/model/10/Format-Serving. ModelRun written to /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/Trainer/model_run/10
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 10
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

## Evaluating trained models with Evaluator
The `Evaluator` component analyzes model performance using the [TensorFlow Model Analysis library](https://www.tensorflow.org/tfx/model_analysis/get_started). It runs inference requests on particular subsets of the test dataset, based on which slices are defined by the developer. Knowing which slices should be analyzed requires domain knowledge of what is important in this particular use case or domain. 

The `Evaluator` can also optionally validate a newly trained model against a previous model. In this lab, you only train one model, so the Evaluator automatically will label the model as "blessed".


<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Evaluator.png width="400">

### Configure and run the Evaluator component


Use the `Resolver` to pick the previous model to compare against.  The model resolver is only required if performing model validation in addition to evaluation. In this case we validate against the latest blessed model. If no model has been blessed before (as in this case) the evaluator will make our candidate the first blessed model.

In [48]:
model_resolver = Resolver(
      #instance_name='latest_blessed_model_resolver',
      strategy_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))

In [49]:
context.run(model_resolver)

INFO:absl:Running driver for Resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for Resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Resolver
    execution_id: 11
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

Configure evaluation metrics and slices.

In [50]:
# TODO: Your code here to create a tfma.MetricThreshold. 
# Review the API documentation here: https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold
# Hint: Review the API documentation for tfma.GenericValueThreshold to constrain accuracy between 50% and 99%.
lower_upper_bound = tfma.GenericValueThreshold(lower_bound={'value':0.5}, #solution
                                               upper_bound={'value':0.99}) #solution

accuracy_threshold =tfma.MetricThreshold(value_threshold=lower_upper_bound)

metrics_specs = tfma.MetricsSpec(
                   metrics = [
                       tfma.MetricConfig(class_name='SparseCategoricalAccuracy',
                           threshold=accuracy_threshold),
                       tfma.MetricConfig(class_name='ExampleCount')])

eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='Cover_Type')
    ],
    metrics_specs=[metrics_specs],
    slicing_specs=[
        tfma.SlicingSpec(),
        tfma.SlicingSpec(feature_keys=['Wilderness_Area'])
    ]
)
eval_config

model_specs {
  label_key: "Cover_Type"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "Wilderness_Area"
}
metrics_specs {
  metrics {
    class_name: "SparseCategoricalAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.5
        }
        upper_bound {
          value: 0.99
        }
      }
    }
  }
  metrics {
    class_name: "ExampleCount"
  }
}

In [51]:
model_analyzer = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config
)

In [52]:
context.run(model_analyzer, enable_cache=False)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "SparseCategoricalAccuracy",\n          "threshold": {\n            "value_threshold": {\n              "lower_bound": 0.5,\n              "upper_bound": 0.99\n            }\n          }\n        },\n        {\n          "class_name": "ExampleCount"\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "Cover_Type"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        "Wilderness_Area"\n      ]\n    }\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': 'null', 'example_splits': 'null', 'module_file': None, 'module_path': None} 'custom_eval_shared_model'
INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is l

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
INFO:absl:Blessing result True written to /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/Evaluator/blessing/12.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 12
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Check the model performance validation status

In [53]:
model_blessing_uri = model_analyzer.outputs['blessing'].get()[0].uri
!ls -l {model_blessing_uri}

total 0
-rw-r--r-- 1 jupyter jupyter 0 Mar  5 16:51 BLESSED


### Visualize evaluation results
You can visualize the evaluation results using the `tfma.view.render_slicing_metrics()` function from TensorFlow Model Analysis library.

**Setup Note**: **Currently, TFMA visualizations don't render in  JupyterLab. Make sure that you run this notebook in Classic Notebook.**

In [54]:
evaluation_uri = model_analyzer.outputs['evaluation'].get()[0].uri
evaluation_uri
!ls {evaluation_uri}

attributions-00000-of-00001.tfrecord  plots-00000-of-00001.tfrecord
eval_config.json		      validations.tfrecord
metrics-00000-of-00001.tfrecord


In [55]:
eval_result = tfma.load_eval_result(evaluation_uri)
eval_result

EvalResult(slicing_metrics=[((('Wilderness_Area', 'Cache'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.5804299712181091}, 'example_count': {'doubleValue': 1349.0}}}}), ((), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.7185328602790833}, 'example_count': {'doubleValue': 20148.0}}}}), ((('Wilderness_Area', 'Commanche'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.7024756073951721}, 'example_count': {'doubleValue': 8806.0}}}}), ((('Wilderness_Area', 'Rawah'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.7676677703857422}, 'example_count': {'doubleValue': 8957.0}}}}), ((('Wilderness_Area', 'Neota'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.6100386381149292}, 'example_count': {'doubleValue': 1036.0}}}})], plots=[((('Wilderness_Area', 'Cache'),), None), ((), None), ((('Wilderness_Area', 'Commanche'),), None), ((('Wilderness_Area', 'Rawah'),), None), ((('Wilderness_Area', 'Neota'),), None)], attributions

In [ ]:
# tfma.view.render_slicing_metrics(eval_result)

In [ ]:
# tfma.view.render_slicing_metrics(
#     eval_result, slicing_column='Wilderness_Area')

## InfraValidator

The `InfraValidator` component acts as an additional early warning layer by validating a candidate model in a sandbox version of its serving infrastructure to prevent an unservable model from being pushed to production. Compared to the `Evaluator` component above which validates a model's performance, the `InfraValidator` component is validating that a model is able to generate predictions from served examples in an environment configured to match production. The config below takes a model and examples, launches the model in a sand-boxed [TensorflowServing](https://www.tensorflow.org/tfx/guide/serving) model server from the latest image in a local docker engine, and optionally checks that the model binary can be loaded and queried before "blessing" it for production.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/InfraValidator.png width="400">

In [ ]:
# infra_validator = InfraValidator(
#     model=trainer.outputs['model'],
#     examples=example_gen.outputs['examples'],
#     serving_spec=infra_validator_pb2.ServingSpec(
#         tensorflow_serving=infra_validator_pb2.TensorFlowServing(
#             tags=['latest']),
#       local_docker=infra_validator_pb2.LocalDockerConfig(),
#   ),
#     validation_spec=infra_validator_pb2.ValidationSpec(
#         max_loading_time_seconds=60,
#         num_tries=5,
#     ),    
#   request_spec=infra_validator_pb2.RequestSpec(
#       tensorflow_serving=infra_validator_pb2.TensorFlowServingRequestSpec(),
#           num_examples=5,
#       )
# )

In [ ]:
!saved_model_cli show --dir "/home/jupyter/serving_model/1741184830" --all

In [65]:
print(dir(tfx.proto.infra_validator_pb2))

['DESCRIPTOR', 'EnvVar', 'EnvVarSource', 'KubernetesConfig', 'LocalDockerConfig', 'PodOverrides', 'RequestSpec', 'Resources', 'SecretKeySelector', 'ServingSpec', 'TensorFlowServing', 'TensorFlowServingRequestSpec', 'ValidationSpec', '_ENVVAR', '_ENVVARSOURCE', '_KUBERNETESCONFIG', '_LOCALDOCKERCONFIG', '_PODOVERRIDES', '_PODOVERRIDES_ANNOTATIONSENTRY', '_REQUESTSPEC', '_RESOURCES', '_RESOURCES_CLAIMSENTRY', '_RESOURCES_LIMITSENTRY', '_RESOURCES_REQUESTSENTRY', '_SECRETKEYSELECTOR', '_SERVINGSPEC', '_TENSORFLOWSERVING', '_TENSORFLOWSERVINGREQUESTSPEC', '_VALIDATIONSPEC', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_builder', '_descriptor', '_descriptor_pool', '_sym_db', '_symbol_database']


In [73]:
infra_validator = InfraValidator(
    model=trainer.outputs['model'],
    examples=example_gen.outputs['examples'],
    serving_spec=infra_validator_pb2.ServingSpec(
        tensorflow_serving=infra_validator_pb2.TensorFlowServing(
            tags=['latest']
        ),
        local_docker=infra_validator_pb2.LocalDockerConfig(),
    ),
    validation_spec=infra_validator_pb2.ValidationSpec(
        max_loading_time_seconds=60,
        num_tries=5,
    )
)

In [77]:
infra_validator = InfraValidator(
    model=trainer.outputs['model'],
    examples=example_gen.outputs['examples'],
    serving_spec=infra_validator_pb2.ServingSpec(
        tensorflow_serving=infra_validator_pb2.TensorFlowServing(
            tags=['latest']
        ),
        local_docker=infra_validator_pb2.LocalDockerConfig(),
    ),
    validation_spec=infra_validator_pb2.ValidationSpec(
        max_loading_time_seconds=60,
        num_tries=5,
    )
)

In [78]:
context.run(infra_validator, enable_cache=False)

INFO:absl:Running driver for InfraValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for InfraValidator
INFO:absl:InfraValidator will be run in LOAD_ONLY mode.
INFO:absl:Creating temp directory at /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/.temp/18/
INFO:absl:Starting infra validation (attempt 1/5).
INFO:absl:Starting LocalDockerRunner(image: tensorflow/serving:latest).
INFO:absl:Running container with parameter {'auto_remove': True, 'detach': True, 'publish_all_ports': True, 'image': 'tensorflow/serving:latest', 'environment': {'MODEL_NAME': 'infra-validation-model', 'MODEL_BASE_PATH': '/model', 'TF_CPP_MAX_VLOG_LEVEL': '3'}, 'mounts': [{'Target': '/model/infra-validation-model/1', 'Source': '/home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/.temp/18/infra-validation-model/1741197008', 'Type': 'bind', 'ReadOnly': True}]}
INFO:absl:Model status is not available yet:
<_InactiveRpcError of RPC that te

ExecutionResult(
    component_id: InfraValidator
    execution_id: 18
    outputs:
        blessing: OutputChannel(artifact_type=InfraBlessing, producer_component_id=InfraValidator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

**Note**: In case of `Infra Validation (attempt 1/5) failed` error, please disregard the error.

### Check the model infrastructure validation status

In [79]:
infra_blessing_uri = infra_validator.outputs['blessing'].get()[0].uri

## Deploying models with Pusher

The `Pusher` component checks whether a model has been "blessed", and if so, deploys it by pushing the model to a well known file destination.

<img src=https://github.com/GoogleCloudPlatform/mlops-on-gcp/raw/master/images/Pusher.png width="400">



### Configure and run the `Pusher` component

In [80]:
trainer.outputs['model']

OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)

In [85]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=model_analyzer.outputs['blessing'],
    infra_blessing=infra_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR)))
context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Model version: 1741197108
INFO:absl:Model written to serving path /home/jupyter/serving_model/1741197108.
INFO:absl:Model pushed to /home/jupyter/artifact-store/tfx-covertype-classifier/20250305_164631/Pusher/pushed_model/20.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 20
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

### Examine the output of `Pusher`

In [82]:
pusher.outputs

{'pushed_model': OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)}

In [83]:
# Set `PATH` to include a directory containing `saved_model_cli.
PATH=%env PATH
%env PATH=/opt/conda/envs/tfx/bin:{PATH}

env: PATH=/opt/conda/envs/tfx/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


In [84]:
print(os.listdir(SERVING_MODEL_DIR))
latest_pushed_model = os.path.join(SERVING_MODEL_DIR, max(os.listdir(SERVING_MODEL_DIR)))
!saved_model_cli show --dir {latest_pushed_model} --all

['1741184830', '1741197089']
2025-03-05 17:51:38.293422: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 17:51:38.299295: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-05 17:51:38.319207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-05 17:51:38.354415: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-05 17:51:38.354507: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 17:51:38.375968: I tensorflow/core/p

**Note**: In case of `FileNotFoundError` error, please disregard the error.

## Next steps

This concludes your introductory walthrough through TFX pipeline components. In the lab, you used TFX to analyze, understand, and pre-process the dataset and train, analyze, validate, and deploy a multi-class classification model to predict the type of forest cover from cartographic features. You utilized a TFX Interactive Context for prototype development of a TFX pipeline directly in a Jupyter notebook. Next, you worked with the TFDV library to modify your dataset schema to add feature constraints to catch data anamolies that can negatively impact your model's performance. You utilized TFT library for feature proprocessing for consistent feature transformations for your model at training and serving time. Lastly, using the TFMA library, you added model performance constraints to ensure you only push more accurate models than previous runs to production.


The next labs in the series will guide through developing a TFX pipeline, deploying and running the pipeline on **AI Platform Pipelines** and automating the pipeline build and deployment processes with **Cloud Build**.

## License

<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>